In [1]:
# Here is where we import all the things we need
# This example is based on Abhishek's Machine Learning Demystified
# https://github.com/shekit/machine-learning-demystified/

# Keras stuff
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
# For if you want to get a file from a URL
# from keras.utils.data_utils import get_file
from keras.optimizers import RMSprop

import numpy as np
# For some random number stuff
import random

Using TensorFlow backend.


In [2]:
# Get text data
# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
path ='data/hamlet.txt'
# read the file and convert to lowercase
# You might want to retain case sensitivity depending on what you are doing
text = open(path).read().lower()  

In [3]:
# How much text is this? (You need a lot for an LSTM to be effective!)
len(text)

173945

In [4]:
# Get a list of all the unique characters
chars = sorted(list(set(text)))
len(chars)

41

In [5]:
# Two dictionaries
# Lookup a character by index
# Lookup an index by character
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [6]:
# Cut your text into overlapping sequences
# Length of each sequence is 40
maxlen = 40

# Each sentence therefore is 40 characters
sentences= []
# With a corresponding "next character"
next_chars=[]

for i in range(0, len(text)-maxlen, 3):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

In [9]:
# Turn all text into vectors (one hot encoding)
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

(57969, 41)

In [39]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]]=1

In [40]:
# build LSTM Model
model = Sequential()
# Create the LSTM layer, 128 units with an input shape of sentence length x number of chars
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# Final stage full-connected layer
model.add(Dense(len(chars)))
# Softmax activiation
model.add(Activation('softmax'))

# Create an optimizer with a learning rate
optimizer = RMSprop(lr=0.01)
# Compile the model with a loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [41]:
# train your model (just one epoch right now)
model.fit(x, y, batch_size=128, epochs=1)

Epoch 1/1
57969/57969 [==============================] - 106s - loss: 2.3151     

In [42]:
# Save the model to generate text later
model.save('model.h5')